pytorch: state_dict()
현재 모델을 "모델상태를 저장한 state_dict"를 이용하여 상태를 설정
torch.nn.Module 객체의 state_dict() 메서드는

모델의 학습 가능한 매개변수(가중치와 바이어스)의 상태와
버퍼(예: BatchNorm의 running mean과 variance 등)의 상태를 저장하는
collections.OrderedDict 객체를 반환

주요 특징
OrderedDict 형태:
state_dict()는
attribute 이름을 키로 하고,
그에 대응하는 torch.Tensor를 값으로 갖는
collections.OrderedDict 객체를 반환.
키로 인덱싱하여 계속 보존, 돌려줌

api..?

In [ ]:
import torch
import torch.nn as nn #얼라이어스함
from collections import OrderedDict

In [ ]:
# 간단한 모델 정의
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()# 모듈 상속시 초기화, 반드시 필요함
        self.linear = nn.Linear(10, 5)
        self.bn = nn.BatchNorm1d(5) # 버퍼
        # self.sub = nn.Sequential(
        #     OrderedDict({
        #         'ds00_layer': nn.Linear(5,5),
        #         'ds00_bn':nn.BatchNorm1d(5),
        #         'ds00_act': nn.ReLU(),
        #         'ds01_layer': nn.Linear(5,5),
        #     })
        # )

    def forward(self, x): #self -인스턴스 메소드일때 사용,
    # forward는 직접 호출하면 절대 안됨
        x = self.linear(x)
        x = self.bn(x)
        return x
        #collable object 검색해보기

In [ ]:
# 모델 인스턴스 생성
model = MyModel()
# 모델의 인스턴스 메소드 호출- state_dict가져옴
state_dict = model.state_dict()

for key, value in state_dict.items():
  print(f"{key}: {value.shape}") #shape을 맞춰야 하기 때문에 출
  # state_dict 내용 출력
#for key, value in state_dict.items():
    #print(f"{key:<25}: {str(type(value)):<20}: {value.shape}")#25글자, 오른쪽으로 정

linear.weight: torch.Size([5, 10])
linear.bias: torch.Size([5])
bn.weight: torch.Size([5])
bn.bias: torch.Size([5])
bn.running_mean: torch.Size([5])
bn.running_var: torch.Size([5])
bn.num_batches_tracked: torch.Size([])


In [ ]:
model.state_dict? #도움말 실

In [ ]:
# state_dict 내용 출력
state_dict0 = model.state_dict(prefix='ds', keep_vars=True) #버터인지 파라미터인지 볼 수 있도록 나타냄
# state_dict 내용 출력
for key, value in state_dict0.items():
  print(f"{key:<25}: {str(type(value)):<20}: {value.shape}")

dslinear.weight          : <class 'torch.nn.parameter.Parameter'>: torch.Size([5, 10])
dslinear.bias            : <class 'torch.nn.parameter.Parameter'>: torch.Size([5])
dsbn.weight              : <class 'torch.nn.parameter.Parameter'>: torch.Size([5])
dsbn.bias                : <class 'torch.nn.parameter.Parameter'>: torch.Size([5])
dsbn.running_mean        : <class 'torch.Tensor'>: torch.Size([5])
dsbn.running_var         : <class 'torch.Tensor'>: torch.Size([5])
dsbn.num_batches_tracked : <class 'torch.Tensor'>: torch.Size([])


keep_vars 는 기본값이 False로 buffers와 parameters 의 값만을 추출할지를 결정
keep_vars=True 인 경우, 값 대신 tensor객체로 데이터 버퍼를 가지고 있는 dictionary가 반환됨.
value 가 파라메터인 경우엔 Parameter 로 얻어지고,
value 가 버퍼인 경우엔 Tensor 로 얻어짐.
keep_vars=True 인 경우, 메모리 사용량이 커지고, 매우 느리고 복잡한 동작이 이루어지지만. 다음의 장점을 가짐.
모델 디버깅: 모델 상태를 조사하고 특정 매개 변수나 버퍼의 값을 변경해야 하는 경우 유용
모델 커스터마이징: 모델을 불러온 후 특정 매개 변수나 버퍼의 값을 변경해야 하는 경우 유용
모델 저장 및 불러오기 확장: 모델 저장 및 불러오기 프로세스를 확장하고 추가적인 정보를 저장해야 하는 경우 유용
하지만, PyTorch의 버전이 정확히 맞아야만 동작할 수 있는 등의 제한점을 가짐.
저장의 용도로는 keep_vars=False를 사용하는 게 좋음.

#활용방법

In [ ]:
#pickle 직렬화시켜 저장 가능
torch.save(model.state_dict(), 'model_state.pth')#state_dict를 파일에 저장

model = MyModel()#모델 로드
model.load_state_dict(torch.load('model_state.pth'))#저장된 state_dict를 로드하여 모델 복원
model.eval()#평가모드로 전환 (선택사항-훈련시는 필요 x)

MyModel(
  (linear): Linear(in_features=10, out_features=5, bias=True)
  (bn): BatchNorm1d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [ ]:
#파라미터 업데이트
state_dict['linear.weight'] = torch.ones_like(state_dict['linear.weight']) #shape맞추기 위해 ones_like사용
model.load_state_dict(state_dict)

<All keys matched successfully>

Module의 상태 확인하기.
.parameters(recurse=True)
optimizer에게 학습을 통해 갱신되어야하는 model의 parameters를 넘겨줄 때 사용됨.

각, 정보를 볼 때는 .named_parameter()를 이용하면, 이름을 같이 확인할 수 있음.

.named_buffers()
buffers는 역시 tensor 객체이나,
학습과정에서 갱신이 필요한 parameters와 달리
학습과정에서 변하지 않는 데이터를 저장하는데 사용된다.



.named_buffers()는 모델 내에 정의된 모든 버퍼를 이름과 함께 dictionary 형식으로 반환함.

#Torch: Save and Load Model


model전체 저장방법: torch.save

모델 저장 방법
모델의 Parameters (= weights and bias)를 저장 (Structure 등은 저장되지 않음).

//
모델의 구조를 정의하고 있는 class 의 instance 코드 상에서 생성하고,
이 instance로 로딩을 수행해줘야 하지만,
해당 class의 소스를 정확히 가지고 있을 경우 PyTorch 버전 등에 상관없이
이전과 동일한 모델을 load를 통해 얻을 수 있음.

load_state_dict( state_dict ) 메서드


현재 Module 인스턴스의 상태를 argument로 넘겨진 OrderedDict 객체 state_dict를 이용 하여 설정함.


이 메서드의 반환값은

torch.nn.modules.module._IncompatibleKeys
Module 인스턴스에 state_dict를 로드할 때 호환되지 않는 키들의 정보 가짐

 2개의 attributes가짐
  -모델의 Parameters를 복원하는데 발생한 문제를 해결하기 위한 조치를 취할 수 있음.

missing_keys : 로드하려는 state_dict에는 있으나 load_state_dict메서드를 호출한 Module 객체에는 없는 키들.

unexpected_keys : Module 객체에는 있으나 인자로 넘겨진 state_dict에는 없는 키들.


#model 전체를 저장하고 로드 방법

state_dict를 이용

In [ ]:
# 필요한 library와 모듈 import
import torch
from torch import nn
from torch.nn import init ##새로 추가됨
from collections import OrderedDict

# 간단한 linear regression model 정의.
# SimpleModel0와 SimpleModel1은
# 똑같은 구조이나 파라메터들의 초기값만 다름.
class SimpleModel0(nn.Module):

  def __init__(self, n_in_f, n_out_f):

    super().__init__()


    init_weigths = torch.ones( (n_in_f, n_out_f) )
    init_bias = torch.zeros( (n_out_f,) )#위 두개는 빠져도 됨

    self.l0 = nn.Linear(n_in_f, n_out_f)

    const_weight = 1.
    const_bias = 0.5

    init.constant_(self.l0.weight, const_weight)#init:텐서 초기화
    #inint_constant:inplace로 처리
    if self.l0.bias is not None: # bias가 있는지 확인
      init.constant_(self.l0.bias, const_bias)

  def forward(self, x):
    return self.l0(x)

class SimpleModel1(nn.Module):

  def __init__(self, n_in_f, n_out_f):

    super().__init__()

    init_weigths = torch.ones( (n_in_f, n_out_f) )
    init_bias = torch.zeros( (n_out_f,) )

    self.l0 = nn.Linear(n_in_f, n_out_f)

    const_weight = 2.
    const_bias = 1.5

    init.constant_(self.l0.weight, const_weight)
    if self.l0.bias is not None:
      init.constant_(self.l0.bias, const_bias)

  def forward(self, x):
    return self.l0(x)




In [ ]:
# 모델 객체를 생성하고, 이에 대한 파라메터 확인후
# 파라메터만 저장.
model = SimpleModel0(3,1)
print(list(model.named_parameters()))
torch.save(model.state_dict(), 'model_params.pth')

# 새로운 모델 객체를 생성.
# 해당 모델 객체는 구조는 같으나, 파라메터들의 초기값은 다름.
n_model = SimpleModel1(3,1)

print('===============')
for old, new in zip(model.parameters(), n_model.parameters()): #zip:묶어줌

  if not torch.equal(old,new):######밑에서 얘기함
    print('model and n_model w/ default init do not have parameters with the same values!')
    break
else:
  print('model and n_model w/ default init have parameters with the same values!')
print('===============') #전부다 같은 값 가짐을 확인

# 이전 저장한 parameters에 대한 state_dict를
# 로드하고 해당 state_dict로 새로만든 모델의
# 파라메터를 설정하고 이전 모델과 비교.
# load parameters and restore old parameters into new model
loaded_params_ordered_dict = torch.load('model_params.pth')
print(f'{type(loaded_params_ordered_dict)=}') # collections.OredredDict

ret_v = n_model.load_state_dict(loaded_params_ordered_dict)
print(f'{type(ret_v)}: {ret_v}')

print('===============')
for old, new in zip(model.parameters(), n_model.parameters()):

  if not torch.equal(old,new):
    print('model and n_model do not have parameters with the same values!')
    break
else:
  print('model and n_model have parameters with the same values!')

[('l0.weight', Parameter containing:
tensor([[1., 1., 1.]], requires_grad=True)), ('l0.bias', Parameter containing:
tensor([0.5000], requires_grad=True))]
model and n_model w/ default init do not have parameters with the same values!
type(loaded_params_ordered_dict)=<class 'collections.OrderedDict'>
<class 'torch.nn.modules.module._IncompatibleKeys'>: <All keys matched successfully>
model and n_model have parameters with the same values!


In [ ]:
# load parameters and restore old parameters into new model
loaded_params_ordered_dict = torch.load('model_params.pth')

incompatible_keys = n_model.load_state_dict(loaded_params_ordered_dict)
print(f'{type(incompatible_keys)}: {incompatible_keys}')
print(f'{incompatible_keys.missing_keys=}')
print(f'{incompatible_keys.unexpected_keys=}')

<class 'torch.nn.modules.module._IncompatibleKeys'>: <All keys matched successfully>
incompatible_keys.missing_keys=[]
incompatible_keys.unexpected_keys=[]


In [ ]:
for old, new in zip(model.parameters(), n_model.parameters()): #old에는 첫번쨰 파라미터, new새로운 파라미

  if not torch.equal(old,new):
    print('model and n_model do not have parameters with the same values!')
    break
else:
  print('model and n_model have parameters with the same values!')

model and n_model have parameters with the same values!


#session을 초기화한 후 다시 수행

class definitiond이 session 초기화로 없어지므로 에러 발생.

In [ ]:
import torch
from torch import nn
from torch.nn import init
from collections import OrderedDict

n_model = torch.load('model.pth')
print(f'{type(n_model)=}, {n_model}')

FileNotFoundError: [Errno 2] No such file or directory: 'model.pth'

In [ ]:

class SimpleModel0(nn.Module):

  def __init__(self, n_in_f, n_out_f):

    super().__init__()

    init_weigths = torch.ones( (n_in_f, n_out_f) )
    init_bias = torch.zeros( (n_out_f,) )

    self.l0 = nn.Linear(n_in_f, n_out_f)

    const_weight = 2.
    const_bias = 1.5

    init.constant_(self.l0.weight, const_weight)
    if self.l0.bias is not None:
      init.constant_(self.l0.bias, const_bias)

  def forward(self, x):
    return self.l0(x)




In [ ]:
n_model = torch.load('model.pth')
print(f'{type(n_model)=}, {n_model}')


FileNotFoundError: [Errno 2] No such file or directory: 'model.pth'

In [ ]:

for c in n_model.named_parameters():
  print(c)


('l0.weight', Parameter containing:
tensor([[1., 1., 1.]], requires_grad=True))
('l0.bias', Parameter containing:
tensor([0.5000], requires_grad=True))


In [ ]:
import pickle
pickle.dump(n_model, open('m', 'pkl', 'wb'))
nn_model = model = pickle.load(open{'m,pkl','rb'})
nn_model

In [ ]:
for old, new in zip(n_model,paraneters(), nn_model.parameters()):

  if not torch.equal(old,new):
    print('model and n_model do not have#...')

#PyTorch: Tensor 비교하기

이 방법은 두 텐서가 완전히 동일한지를 확인.

1. torch.equal 사용

이 방법은 두 텐서가 완전히 동일한지를 확인.

In [ ]:
import torch
import torch.nn as nn

# nn.Parameter 객체 생성
param1 = nn.Parameter(torch.tensor([1.0, 2.0, 3.0]))
param2 = nn.Parameter(torch.tensor([1.0, 2.0, 3.0]))
param3 = nn.Parameter(torch.tensor([1.0, 2.0, 4.0]))

# 값 비교
print(torch.equal(param1, param2))  # True
print(torch.equal(param1, param3))  # False

2. torch.allclose 사용

이 방법은 두 텐서가 지정된 허용 오차 내에서 거의 동일한지를 확인.

이는 부동 소수점 연산의 미세한 차이로 인한 불일치를 허용할 수 있음 (권장).

In [ ]:
import torch
import torch.nn as nn

# nn.Parameter 객체 생성
param1 = nn.Parameter(torch.tensor([1.0, 2.0, 3.0]))
param2 = nn.Parameter(torch.tensor([1.0, 2.0, 3.0000001]))
param3 = nn.Parameter(torch.tensor([1.0, 2.0, 4.0]))

# 값 비교
print(torch.allclose(param1, param2))  # True, 기본 허용 오차 내
print(torch.allclose(param1, param3))  # False

2-1. torch.allclose의 허용 오차 설정

허용 오차를 설정하여 두 텐서가 거의 동일한지 확인할 수 있음.

In [ ]:
import torch
import torch.nn as nn

# nn.Parameter 객체 생성
param1 = nn.Parameter(torch.tensor([1.0, 2.0, 3.0]))
param2 = nn.Parameter(torch.tensor([1.0, 2.0, 3.0001]))

# 값 비교, 허용 오차 설정
print(torch.allclose(param1, param2, atol=1e-4))  # True, 허용 오차 내
print(torch.allclose(param1, param2, atol=1e-5))  # False, 허용 오차 밖

설명


torch.allclose(tensor1, tensor2, rtol=1e-05, atol=1e-08):
두 텐서가 상대적(rtol) 및 절대적(atol) 허용 오차 내에서 거의 동일한지 확인.

# PyTorch: nn.ModuleList, nn.ModuleDict, nn.Sequential

1. nn.Sequential

여러 모듈을 순차적으로 실행할 수 있도록 하는 PyTorch 클래스

간단한 네트워크 구조를 만들 때 유용하며, 모듈을 정의한 순서대로 순차적으로 적용

In [ ]:
import torch
import torch.nn as nn

model = nn.Sequential(
    nn.Linear(10, 20),
    nn.ReLU(),
    nn.Linear(20, 10)
)

print(model)

Sequential(
  (0): Linear(in_features=10, out_features=20, bias=True)
  (1): ReLU()
  (2): Linear(in_features=20, out_features=10, bias=True)
)


=>nn.Sequential은 nn.Linear와 nn.ReLU 모듈을 순차적으로 포함하고 있으며, 정의한 순서대로 순차적으로 적용함.

nn.Sequential을 사용하면 네트워크 구조를 직관적으로 정의

2. nn.ModuleList

PyTorch의 nn.Module을 리스트처럼 다루기 위한 클래스
리스트에 포함된 모든 모듈이 PyTorch의 모델 구성 요소로 인식되어 올바르게 등록되고 관리됨.

이는 모델 파라미터가 자동으로 추적되며, to(), cuda(), cpu()와 같은 메서드를 사용할 수 있게 함.

In [ ]:
import torch
import torch.nn as nn

class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.layers = nn.ModuleList([ #꼭 묶어 주어야 함
            nn.Linear(10, 20),
            nn.ReLU(),
            nn.Linear(20, 10)
        ])

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

model = MyModel()
print(model)

MyModel(
  (layers): ModuleList(
    (0): Linear(in_features=10, out_features=20, bias=True)
    (1): ReLU()
    (2): Linear(in_features=20, out_features=10, bias=True)
  )
)


=>리스트 안에 nn.Linear와 nn.ReLU 모듈을 포함

forward 메서드에서는 리스트에 포함된 각 모듈을 순차적으로 적용

3. nn.ModuleDict (이걸 권함)

PyTorch의 nn.Module을 딕셔너리처럼 다루기 위한 클래스
딕셔너리에 포함된 모든 모듈이 PyTorch의 모델 구성 요소로 인식되어 올바르게 등록되고 관리됨.

이는 모델 파라미터가 자동으로 추적되며, to(), cuda(), cpu()와 같은 메서드를 사용할 수 있게 함.

In [ ]:
import torch
import torch.nn as nn

class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.layers = nn.ModuleDict({
            'fc1': nn.Linear(10, 20),
            'relu': nn.ReLU(),
            'fc2': nn.Linear(20, 10)
        })

    def forward(self, x):
        x = self.layers['fc1'](x)
        x = self.layers['relu'](x)
        x = self.layers['fc2'](x)
        return x

model = MyModel()
print(model)

MyModel(
  (layers): ModuleDict(
    (fc1): Linear(in_features=10, out_features=20, bias=True)
    (relu): ReLU()
    (fc2): Linear(in_features=20, out_features=10, bias=True)
  )
)


=> 키-값 쌍으로 nn.Linear와 nn.ReLU 모듈을 포함하고 있음.

forward 메서드에서는 딕셔너리의 키를 사용하여 각 모듈을 순차적으로 적용함.



## 일반 list나 dict를 사용할 때 발생하는 문제

일반 list나 dict를

nn.ModuleList나 nn.ModuleDict 대신 사용하면 여러 가지 문제가 발생

**문제점은 list 또는 dict 내의 모듈들의
파라메터 자동 추적 이 이루어지지 않는 다는 점임.

PyTorch는

nn.ModuleList나 nn.ModuleDict에 포함된 모듈의 파라미터를 자동으로 추적함.

이를 통해 모델의 모든 파라미터가 자동으로 등록되고, parameters() 메서드을 통해 optim 등에서 쉽게 접근

In [ ]:
##문제점 확인 코드
import torch
import torch.nn as nn

class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.layers = [
            nn.Linear(10, 20),
            nn.ReLU(),
            nn.Linear(20, 10)
        ]

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

model = MyModel()
print(model)
print("Model parameters:", list(model.parameters()))  # 파라미터가 추적되지 않음
#텅 빈 형태로 출

MyModel()
Model parameters: []


**이는 GPU/CPU 이동할 때에도 문제가 됨

nn.ModuleList나 nn.ModuleDict를 사용하면,

model.to('cuda')나 model.cuda()와 같은 메서드를 호출할 때 포함된 모든 모듈이 자동으로 GPU로 이동하지만,

list나 dict를 사용하면 이 동작이 자동으로 이루어지지 않음.


In [ ]:
#문제점 확인 코
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.layers = [
            nn.Linear(10, 20),
            nn.ReLU(),
            nn.Linear(20, 10)
        ]

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

model = MyModel()
model_gpu = model.to('cuda')  # 일반 list를 사용하면 GPU로 이동하지 않음

PyTorch 모델을 구성할 때는 일반 list나 dict가 아닌 nn.ModuleList, nn.ModuleDict, nn.Sequential를 사용해야 한다.

달리 애기하면 top-level attribute 로 sub-module을 추가하는 경우 외에는 nn.ModuleList, nn.ModuleDict, nn.Sequential를 사용해야 한다.